# LOGISTIC REGGRESION : TITANIC DATASET

*   Predict what passengers survived the titanic crash based only on passenger's features (age, cabin, children, etc...)
*   **PIPELINES**
*   Missing Informations
*   Categorical columns

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('titanic').getOrCreate()

23/08/15 20:19:50 WARN Utils: Your hostname, Blade-15-Base-Model resolves to a loopback address: 127.0.1.1; using 192.168.1.231 instead (on interface wlo1)
23/08/15 20:19:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/15 20:19:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.csv('titanic.csv', inferSchema=True, header=True)

In [4]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [5]:
df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [6]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [7]:
my_cols = df.select(['Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked'])

In [8]:
my_final_data = my_cols.na.drop()

In [9]:
my_final_data.show()

+--------+------+------+----+-----+-----+-------+--------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|
+--------+------+------+----+-----+-----+-------+--------+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|
|       1|     1|female|38.0|    1|    0|71.2833|       C|
|       1|     3|female|26.0|    0|    0|  7.925|       S|
|       1|     1|female|35.0|    1|    0|   53.1|       S|
|       0|     3|  male|35.0|    0|    0|   8.05|       S|
|       0|     1|  male|54.0|    0|    0|51.8625|       S|
|       0|     3|  male| 2.0|    3|    1| 21.075|       S|
|       1|     3|female|27.0|    0|    2|11.1333|       S|
|       1|     2|female|14.0|    1|    0|30.0708|       C|
|       1|     3|female| 4.0|    1|    1|   16.7|       S|
|       1|     1|female|58.0|    0|    0|  26.55|       S|
|       0|     3|  male|20.0|    0|    0|   8.05|       S|
|       0|     3|  male|39.0|    1|    5| 31.275|       S|
|       0|     3|female|14.0|    0|    0| 7.8542|       

In [18]:
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder, VectorIndexer

# Explications : 
*   **StringIndexer** : allows us to convert every string into a number 

for example : A B C D -> 0 1 2 3 

*   **OneHotEncoder** : transforms the actual numbers for the categories into a one hot encoding : an arrays with 0s and 1s

One hot Encoding of a key : 

A -> [1 0 0 0] 

B -> [0 1 0 0]

C -> [0 0 1 0]

D -> [0 0 0 1] 

In [29]:
gender_indexer = StringIndexer(inputCol='Sex',outputCol='SexIndex') 
gender_incoder = OneHotEncoder(inputCol='SexIndex', outputCol='SexVec')

In [30]:
embark_indexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkIndex') 
embark_incoder = OneHotEncoder(inputCol='EmbarkIndex', outputCol='EmbarkVec')

In [31]:
assembler = VectorAssembler(inputCols=['Pclass','SexVec','EmbarkVec','Age','SibSp','Parch','Fare'],
                            outputCol='features')

# PIPELINE

In [32]:
from pyspark.ml.classification import LogisticRegression

In [33]:
from pyspark.ml import Pipeline

In [34]:
log_reg_titanic = LogisticRegression(featuresCol='features', labelCol='Survived')

In [35]:
pipeline = Pipeline(stages=[gender_indexer,embark_indexer,
                            gender_incoder,embark_incoder,
                            assembler,log_reg_titanic])

# stages is a list of the things we gonna do to the data

In [36]:
train_data , test_data = my_final_data.randomSplit([0.7,0.3])

In [37]:
fit_model = pipeline.fit(train_data)

In [38]:
results = fit_model.transform(test_data)

In [41]:
results.show()

+--------+------+------+----+-----+-----+--------+--------+--------+-----------+-------------+-------------+--------------------+--------------------+--------------------+----------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|    Fare|Embarked|SexIndex|EmbarkIndex|       SexVec|    EmbarkVec|            features|       rawPrediction|         probability|prediction|
+--------+------+------+----+-----+-----+--------+--------+--------+-----------+-------------+-------------+--------------------+--------------------+--------------------+----------+
|       0|     1|female|25.0|    1|    2|  151.55|       S|     1.0|        0.0|    (1,[],[])|(2,[0],[1.0])|[1.0,0.0,1.0,0.0,...|[-2.7161415349070...|[0.06202757335806...|       1.0|
|       0|     1|  male|18.0|    1|    0|   108.9|       C|     0.0|        1.0|(1,[0],[1.0])|(2,[1],[1.0])|[1.0,1.0,0.0,1.0,...|[-0.8299599617239...|[0.30365353612565...|       1.0|
|       0|     1|  male|19.0|    3|    2|   263.0|       S|     0.0|        0.0|(1,[0

# EVALUATION

In [39]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [40]:
eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Survived')

In [43]:
AUC = eval.evaluate(results)

In [44]:
AUC

0.8003095975232198